In [52]:
from __future__ import print_function

In [53]:
import sys
sys.path.append('../build/')

In [54]:
%pylab inline
np.set_printoptions(precision=4, suppress=True)

Populating the interactive namespace from numpy and matplotlib


In [55]:
import versor as vsr

## Dataset generation

In [56]:
M_object_in_world = vsr.Vec(1.0,1.0,0.1).trs() * vsr.Biv(pi/4.0, 0.0,0.0).exp()
M_eye_in_hand = vsr.Vec(-0.1, -0.2, 0.3).trs() * (vsr.Biv(3.0,-1.0,4.0).unit() * (pi/9)).exp()
M_eye_in_hand_initial = vsr.Vec(0.11, 0.24, 0.36).trs() * (vsr.Biv(1.0,1.0,0.0).unit() * (np.pi/5.)).exp()

In [57]:
def hand_eye_pose_pairs(M_object_in_world, M_eye_in_hand, n):
    pose_pairs = [
            (M_hand_in_world, 
             (M_eye_in_hand.rev() * M_hand_in_world.rev() * M_object_in_world))
            for M_hand_in_world in [vsr.Vec(*np.random.rand(3)).trs() *
                                    vsr.Rot(vsr.Biv(*np.random.rand(3)).unit() *
                                            np.random.rand() * np.pi)
                                    for i in range(n)]]

    As = [pose_pair[0] for pose_pair in pose_pairs]
    Bs = [pose_pair[1] for pose_pair in pose_pairs]

    LAs = []
    LBs = []
    for i in range(n):
        for j in range(i+1,n):
            LAs.append(((As[j].rev() * As[i]).log() * 0.5).unit())
            LBs.append(((Bs[j] * Bs[i].rev()).log() * 0.5).unit())
    return LAs, LBs

In [58]:
n_lines = 10
lines_a, lines_b = hand_eye_pose_pairs(M_object_in_world, M_eye_in_hand, n_lines)
print(len(lines_a))

45


## Hand-Eye Solver

In [59]:
def daniilidis_motor(LAs, LBs):
    
    def Skew(a):
        return np.array([[0, -a[2], a[1]],
                         [a[2], 0, -a[0]],
                         [-a[1], a[0], 0]])
    
    Ds = []
    for LA, LB in zip(LAs, LBs):
        D = np.zeros((6,8))
        a_ = LA.biv()
        ap_ = LA.drv().vec().unduale()
        b_ = LB.biv()
        bp_ = LB.drv().vec().unduale()
        D[:3,0] = np.array(b_ - a_)
        D[:3,1:4] = Skew(b_ + a_)
        D[3:,0] = np.array(bp_ - ap_)
        D[3:,1:4] = Skew(bp_ + ap_)
        D[3:,4:] = D[:3,:4]
        Ds.append(D[1:7,:].copy())
    
    Ds = np.array(Ds).reshape(-1,8)
    [U, s, Vt] = np.linalg.svd(Ds)

    v7 = Vt.T[:,-2].copy()
    v8 = Vt.T[:,-1].copy()
    
    u1 = v7[:4]
    v1 = v7[4:]
    u2 = v8[:4]
    v2 = v8[4:]

    a = np.inner(u1,v1)
    b = np.inner(u1,v2) + np.inner(u2,v1)
    c = np.inner(u2,v2)
    [s1, s2] = np.roots([a,b,c])

    val1 = (s1**2 * np.inner(u1,u1)) + (2 * s1 * np.inner(u1,u2)) + (np.inner(u2,u2))
    val2 = (s2**2 * np.inner(u1,u1)) + (2 * s2 * np.inner(u1,u2)) + (np.inner(u2,u2))
    
    if val1 > val2:
        s = s1
        val = val1
    else:
        s = s2
        val = val2

    lambda2 = np.sqrt(1.0/val)
    lambda1 = s * lambda2
    
    m = lambda1 * Vt.T[:,-2] + lambda2 * Vt.T[:,-1]
    s = m[0]
    A = vsr.Biv(m[1], m[2], m[3])
    B = vsr.Biv(m[5], m[6], m[7]).duale()
    s4 = -m[4]
    
    mot = vsr.Mot(s, A[0], A[1], A[2], B[0], B[1], B[2], s4)

    return mot * -1

## Experiments

In [60]:
M_eye_in_hand

Mot: [ 0.94 -0.2 0.067 -0.27 0.077 0.044 -0.16 0.01 ]

In [61]:
m = daniilidis_motor(lines_a, lines_b)
print(M_eye_in_hand)
print(m)

Mot: [ 0.94 -0.2 0.067 -0.27 0.077 0.044 -0.16 0.01 ]
Mot: [ -0.94 0.2 -0.067 0.27 -0.077 -0.044 0.16 -0.01 ]
